<a href="https://colab.research.google.com/github/BrunoGomesCoelho/mosquito-networking/blob/master/notebooks/0.9-BrunoGomesCoelho-Actual-Colab%20experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
TESTING = True
COLAB = True

if COLAB:
    BASE_DIR = "/content/drive/My Drive/IC/mosquito-networking/"
else:
    BASE_DIR = "../"

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append("/content/drive/My Drive/IC/mosquito-networking/")

In [4]:
!python3 -m pip install -qr "/content/drive/My Drive/IC/mosquito-networking/drive_requirements.txt"

     |████████████████████████████████| 2.1MB 2.7MB/s 
     |████████████████████████████████| 71kB 24.1MB/s 
     |████████████████████████████████| 122kB 43.0MB/s 
     |████████████████████████████████| 51kB 19.5MB/s 
     |████████████████████████████████| 5.7MB 39.5MB/s 
     |████████████████████████████████| 61kB 21.7MB/s 
     |████████████████████████████████| 51kB 20.7MB/s 




- - - 

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Trying out a full pytorch experiment, with tensorboard, // processing, etc

In [0]:
# OPTIONAL: Load the "autoreload" extension so that code can change
#%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
#%autoreload 2

import numpy as np
import pandas as pd
from src.data import make_dataset
from src.data import read_dataset
from src.data import util
from src.data.colab_dataset import MosquitoDatasetColab
import joblib

from torchsummary import summary

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Experiment params

In [0]:
# Parameters
params = {'batch_size': 32,
          'shuffle': True,
          'num_workers': 0}
max_epochs = 1

if TESTING:
    params["num_workers"] = 0

version = !python3 --version
version = version[0].split(".")[1]

if int(version) < 7 and params["num_workers"]:
    print("WARNING\n"*10)
    print("Parallel execution only works for python3.7 or above!")
    print("Running in parallel with other versions is not guaranted to work")
    print("See https://discuss.pytorch.org/t/valueerror-signal-number-32-out-of-range-when-loading-data-with-num-worker-0/39615/2")

In [7]:
## Load gpu or cpu
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

Using device cuda:0


# load data

In [0]:
# Load scaler
#scaler = joblib.load("../data/interim/scaler.pkl")
scaler = joblib.load(BASE_DIR + "data/interim/scaler.pkl")

In [0]:
data = np.load(BASE_DIR + "data/interim/all_wavs.npy", allow_pickle=True)
data = data[data[:, -1].argsort()]

In [0]:
df = pd.read_csv(BASE_DIR + "data/interim/file_names.csv")
df.sort_values("original_name", inplace=True)

In [0]:
errors = (df["original_name"].values != data[:, -1]).sum()
if errors:
    print(f"We have {errors} errors!")
    raise ValueError("Error in WAV/CSV")

In [0]:
x = data[:, 0]
y = df["label"]

train_idx = df["training"] == 1

In [0]:
# Generators
training_set = MosquitoDatasetColab(x[train_idx], y[train_idx].values,
                                    device=device, scaler=scaler)
training_generator = torch.utils.data.DataLoader(training_set, **params, 
                                                 pin_memory=True)

test_set = MosquitoDatasetColab(x[~train_idx], y[~train_idx].values,
                                device=device, scaler=scaler)

                                   
test_generator = torch.utils.data.DataLoader(test_set, **params, 
                                             pin_memory=True)

#sc Generate some example data
temp_generator = torch.utils.data.DataLoader(training_set, **params)

for (local_batch, local_labels) in temp_generator:
    example_x = local_batch
    example_y = local_labels
    break


[1D conv in Pytorch](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv1d) 

In the simplest case, the output value of the layer with input size

$$ (N, C_{\text{in}}, L) $$ and output $$ (N, C_{\text{out}}, L_{\text{out}}) $$ can be

$$ (N, C_{\text{in}}, L) $$ and output $$ (N, C_{\text{out}}, L_{\text{out}}) $$ can be
    precisely described as:

$$
        \text{out}(N_i, C_{\text{out}_j}) = \text{bias}(C_{\text{out}_j}) +
        \sum_{k = 0}^{C_{in} - 1} \text{weight}(C_{\text{out}_j}, k)
        \star \text{input}(N_i, k)
$$ 

where $$ \star $$  is the valid "cross-correlation"  operator,
    N is a batch size, C denotes a number of channels,
    L is a length of signal sequence.

# Load model

In [0]:
from src.models.BasicMosquitoNet import BasicMosquitoNet

In [0]:
# create your optimizer
net = BasicMosquitoNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

if device.type == "cuda":
    net.cuda()

In [16]:
summary(net, input_size=example_x.shape[1:])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1            [-1, 100, 7309]          12,900
            Conv1d-2            [-1, 100, 3527]       1,280,100
            Linear-3                    [-1, 1]         176,301
Total params: 1,469,301
Trainable params: 1,469,301
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.08
Forward/backward pass size (MB): 8.27
Params size (MB): 5.60
Estimated Total Size (MB): 13.96
----------------------------------------------------------------


# Start tensorboard

Run in our terminal: 

`cd notebooks`

`tensorboard --logdir runs`

In [0]:
from torch.utils.tensorboard import SummaryWriter
    
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter()

# train function

In [0]:
# Simple train function

def train(net, optimizer, max_epochs, testing=False, testing_idx=0):
    # Loop over epochs
    last_test_loss = 0
    for epoch in range(max_epochs):
        # Training
        for idx, (local_batch, local_labels) in enumerate(training_generator):
            local_batch, local_labels = util.convert_cuda(local_batch, 
                                                         local_labels,
                                                         device)

            optimizer.zero_grad()   # zero the gradient buffers
            output = net(local_batch)
            loss = criterion(output, local_labels)
            loss.backward()
            optimizer.step()    # Does the update
            
            if testing and idx == testing_idx:
                    break
        
        #writer.add_scalar("Train Loss Batch", loss.data.item(), idx)

        # Validation
        with torch.set_grad_enabled(False):
            cumulative_test_loss = 0
            
            for idx, (local_batch, local_labels) in enumerate(training_generator):
                local_batch, local_labels = util.convert_cuda(local_batch, 
                                                 local_labels,
                                                 device)
                output = net(local_batch)
                loss = criterion(output, local_labels)
                if testing:
                    break
                else:
                    #writer.add_scalar("Test Loss Batch", loss.data.item(), idx)
                    pass
                
                cumulative_test_loss += loss.data.item()
            cumulative_test_loss /= (idx+1)
            last_test_loss = cumulative_test_loss
            
            if not testing:
                pass
                #writer.add_scalar("Test Loss Epoch", loss.data.item(), idx)
    
    return last_test_loss
    writer.close()

In [19]:
%%time
train(net, optimizer, 1, testing=TESTING)

CPU times: user 355 ms, sys: 232 ms, total: 587 ms
Wall time: 638 ms


0.0

In [0]:
# Save the model
#torch.save(net.state_dict(), "../models/0.6-BrunoGomesCoelho-test-experiment.pt")

In [21]:
import time
start_time = time.time()
train(net, optimizer, 1, testing=False)

0.6931632295105279

In [22]:
print(time.time()-start_time)

470.46156334877014


In [24]:
# So we have ~8 min = 470 / 60 = 
470 / 60

7.833333333333333